# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

The assignment I received only came with this instruction:
`Your job as a data scientist is in this assignment is to examine a set of job postings with salaries and then predict salaries for a new set of job postings.`

I didn't receive any additional background or information about why I'd be asked to do this. Understanding the why behind any data science project is crucial, so I decided to come up with a plausible explanation.

I've received this data about job postings from someone in HR. They would like a model that can calculate a suggested salary range for any new position opened up at the company. Any salary offered outside of this band would be subject to additional approvals before it could be offered to a candidate.

In [ ]:
#import your libraries
import pandas as pd
#import sklearn as sk
#etc

#your info here
__author__ = "Steve Anderson"
__email__ = "steve@ranksmarts.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [ ]:
train_feat = pd.read_csv('data/train_features.csv')
train_feat.head(3)

In [ ]:
train_salaries = pd.read_csv('data/train_salaries.csv')
train_salaries.head(3)

In [ ]:
test_feat = pd.read_csv('data/test_features.csv')
test_feat.head(3)

### ---- 3 Clean the data ----

In [ ]:
#look for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it

### ---- 4 Explore the data (EDA) ----

In [ ]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [ ]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [ ]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [ ]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [ ]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data